In [ ]:
%matplotlib nbagg
import matplotlib.pyplot as plt
from IPython.display import display
##
import json
import re
import numpy           as np
import pandas          as pd
import statsmodels.api as sm
import ipywidgets      as widgets
##
import thundermint.logs  as Log
import thundermint.plot  as plot
import thundermint.splot as splot

In [ ]:
logs = Log.load_logs_files('../thundermint/logs/')
p    = plot.CommitData(logs)

In [ ]:
class Tabs(object):
    def __init__(self, names):
        self.names = names
    def __enter__(self):
        outs = [widgets.Output() for _ in self.names]
        tab  = widgets.Tab(children=outs)
        for i,nm in enumerate(self.names):
            tab.set_title(i,nm)
        display(tab)
        return outs
    def __exit__(self, type, value, trace):
        pass

# Block and commit information

In [ ]:
with Tabs(["Height", "Residuals", "Round"]) as tabs:
    with tabs[0]:
        with plot.LegendPlot() as ax:
            p.plot_points(ax, reltime=True)
    with tabs[1]:
        with plot.LegendPlot() as ax:
            p.plot_residuals_TvsH(ax)
    with tabs[2]:
        plot.plot_round(logs)

In [ ]:
with Tabs(["Block size", "Block size (hist)", "N sigs"]) as tabs:
    with tabs[0]:
        with plot.SimplePlot() as ax:
            p.plot_ntx(ax)
    with tabs[1]:
        with plot.SimplePlot() as ax:
            p.plot_ntx_distr(ax)
    with tabs[2]:
        with plot.SimplePlot() as ax:
            p.plot_n_signatures(ax)

# Mempool

In [ ]:
df = logs['node-1.log']
with Tabs(['mempool size', 'filtering time', 'filtering per tx']) as tabs:
    with tabs[0]:
        plot.plot_mempool_size(logs)
    with tabs[1]:
        plt.figure()
        plt.grid()
        plt.title("Filtering time")
        plt.hist(df.mempoolStat['fltT'], bins=50)
    with tabs[2]:
        plt.figure()
        plt.grid()
        plt.title("Filtering time")
        plt.hist(df.mempoolStat['fltT'] / df.mempoolStat['sizeB'], bins=50)

# Timing of algorithm steps

In [ ]:
splot.splot(logs, w=8096)

In [ ]:
with Tabs(Log.keyOrdSet) as tabs:
    for i,k in enumerate(Log.keyOrdSet):
        v = logs['node-1.log'].stepsTime[k]
        with tabs[i]:
            plt.figure()
            plt.grid()
            plt.title(k)
            plt.hist(v['dt'].values.astype('float')/1e6, bins=100)

# Playground

In [ ]:
plt.figure()
plt.grid()
plt.semilogy()
ys,xs,_ = plt.hist(logs['node-1.log'].stepsTime['Entering new height ----------------']['dt'].values.astype('float')/1e6, bins=100)
ys = np.log(ys)
xs = (xs + (xs[1] - xs[0])/2)[:-1]
xs = xs[ys>0]
ys = ys[ys>0]
rr = sm.OLS(ys, sm.add_constant(xs), missing="drop").fit()
plt.plot(xs, np.exp((xs*rr.params[1] + rr.params[0])))
plt.title('Entering new height. N~exp(-t/%.3g)' % (-1/rr.params[1]))
None